<a href="https://colab.research.google.com/github/Pitch2342/Rand/blob/master/Dataset_Titanic/TitanicPred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Titatic dataset

importing libraries

In [0]:
import numpy as np
import pandas as pd
import math
import random

Getting csv

In [2]:
data_train = pd.read_csv('https://raw.githubusercontent.com/Pitch2342/Rand/master/Dataset_Titanic/train.csv')
data_test = pd.read_csv('https://raw.githubusercontent.com/Pitch2342/Rand/master/Dataset_Titanic/test.csv')
data_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


ML begins

In [3]:
def get_x(data):
    features = data[['Pclass', 'Sex', 'Age', 'Fare']].copy()
    
    features['RelativesCount'] = data['SibSp'] + data['Parch']
    
    # impute age using median
    age_std = data_train['Age'].std()
    age_median = data_train['Age'].median()
    features['Age'] = features[['Age']].fillna(method='ffill').astype(int)

    # impute fare using median
    fare_impute_val = data_train['Fare'].median()
    features['Fare'] = data[['Fare']].fillna(method='ffill').astype(int)
    
#     # impute embarked as most mode
#     embarked_impute_val = data_train.Embarked.mode()
#     features.Embarked = features.Embarked.fillna(value=embarked_impute_val)

#     features['RelativesCount'] = features.SibSp + features.Parch
    
    # one hot encodes categorical features
    features = pd.get_dummies(features, columns=['Pclass', 'Sex'])
    
    return features

get_x(data_train).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
Age               891 non-null int64
Fare              891 non-null int64
RelativesCount    891 non-null int64
Pclass_1          891 non-null uint8
Pclass_2          891 non-null uint8
Pclass_3          891 non-null uint8
Sex_female        891 non-null uint8
Sex_male          891 non-null uint8
dtypes: int64(3), uint8(5)
memory usage: 25.4 KB


In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

X_train, X_test, y_train, y_test = train_test_split(
    get_x(data_train), 
    data_train['Survived'], 
    test_size=0.33, 
    random_state=42
)

# Fit model
# logistic reg
# params = {
#     'penalty': ['l1', 'l2']
# }
# model = GridSearchCV(estimator=LogisticRegression(solver='liblinear', random_state=18), param_grid=params, cv=5)

# random forest
params = {
    'n_estimators': [100]
#     'min_samples_leaf': [1, 2, 3],
#     'min_samples_split': [ 2, 3, 4]
}
model = GridSearchCV(estimator=RandomForestClassifier(random_state=18), param_grid=params, cv=5)

# params = {
#     'learning_rate': [0.2, 0.4, 0.6, 0.8, 1],
#     'loss': ['deviance', 'exponential']
# }
# model = GridSearchCV(estimator=GradientBoostingClassifier(random_state=18), param_grid=params, cv=5)

model.fit(X_train, y_train)

# Test fit
y_pred_prob = model.predict_proba(X_test)[:,1]
print("ROAUC: {:0.4f}".format(roc_auc_score(y_test, y_pred_prob)))
print("Score: {:0.4f}".format(model.score(X_test, y_test)))
print(pd.DataFrame(data={'feature': X_test.columns, 'importance': model.best_estimator_.feature_importances_}).sort_values(by=['importance'], ascending=False))

ROAUC: 0.8765
Score: 0.8034
          feature  importance
0             Age    0.318779
1            Fare    0.215761
7        Sex_male    0.142583
6      Sex_female    0.135394
2  RelativesCount    0.080417
5        Pclass_3    0.060232
3        Pclass_1    0.024519
4        Pclass_2    0.022314


In [11]:
# TODO should I re-train with all training data? Found retraining on all data lowered score
# Fit model with all training data
model.fit(
    get_x(data_train), 
    data_train['Survived']
)

# Predict survivors
test_y = model.predict(get_x(data_test))

# Write output
output = data_test[['PassengerId']].copy()
output['Survived'] = test_y
output.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,1
3,895,1
4,896,1


Compare with actual data. import that first

In [6]:
data_result = pd.read_csv('https://raw.githubusercontent.com/Pitch2342/Rand/master/Dataset_Titanic/gender_submission.csv')
data_result

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [7]:
output.where(output.values==data_result.values)

,PassengerId,Survived
0,892,0.0
1,893,NaN
2,894,NaN
3,895,NaN
4,896,1.0
...,...,...
413,1305,0.0
414,1306,1.0
415,1307,0.0
416,1308,0.0


In [22]:
def getAndPred ( PassengerId , Pclass , Name , Sex , Age , SibSp , Parch , Ticket , Fare , Cabin , Embarked = 'S' ):
  data_test.loc[1] = [PassengerId , Pclass , Name , Sex , Age , SibSp , Parch , Ticket , Fare , Cabin , Embarked ]
  A = data_test
  print(A)
  print(get_x(A))
  """Result = model.predict(get_x(A))
  print(Result)"""
  # Predict survivors
  test_y = model.predict(get_x(A))
  #print(A.ix[1])
  if test_y[1] == 1:
    print("Survived")
  else:
    print("Dead")
getAndPred(1,1,"Valencia DiCortez","female",3,5,5,'C1',10.23,'C10','Q')

     PassengerId  Pclass  ... Cabin Embarked
0            892       3  ...   NaN        Q
1              1       1  ...   C10        Q
2            894       2  ...   NaN        Q
3            895       3  ...   NaN        S
4            896       3  ...   NaN        S
..           ...     ...  ...   ...      ...
413         1305       3  ...   NaN        S
414         1306       1  ...  C105        C
415         1307       3  ...   NaN        S
416         1308       3  ...   NaN        S
417         1309       3  ...   NaN        C

[418 rows x 11 columns]
     Age  Fare  RelativesCount  ...  Pclass_3  Sex_female  Sex_male
0     34     7               0  ...         1           0         1
1      3    10              10  ...         0           1         0
2     62     9               0  ...         0           0         1
3     27     8               0  ...         1           0         1
4     22    12               2  ...         1           1         0
..   ...   ...             